## 測試項目
1. 訓練集中的alarm聲
2. 使用
3. export to cc file

In [2]:
import os
import sys
import datetime
import shutil
import numpy as np
import tensorflow as tf
# from tensorflow import keras
import zipfile
import wavio
from common import utils as U

2024-01-10 17:09:57.449781: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-10 17:09:57.469909: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-10 17:09:57.469930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-10 17:09:57.470451: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-10 17:09:57.473963: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from datetime import datetime
import re

### Loading TFlite Model

In [4]:
tflite_quant_model_path = "./models/tflite/acdnet_model_acc96_55_prunned20240110135513.tflite";
# Load quantized TFLite model
tflite_interpreter_quant = tf.lite.Interpreter(model_path=tflite_quant_model_path)

In [5]:
# model.summary()
input_details = tflite_interpreter_quant.get_input_details()
output_details = tflite_interpreter_quant.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

#resize the input and output
# tflite_interpreter_quant.resize_tensor_input(input_details[0]['index'], (2, 1, 30225, 1))
# tflite_interpreter_quant.resize_tensor_input(output_details[0]['index'], (2, 2))

# input_details = tflite_interpreter_quant.get_input_details()
# output_details = tflite_interpreter_quant.get_output_details()

# print("== Input details ==")
# print("name:", input_details[0]['name'])
# print("shape:", input_details[0]['shape'])
# print("type:", input_details[0]['dtype'])

# print("\n== Output details ==")
# print("name:", output_details[0]['name'])
# print("shape:", output_details[0]['shape'])
# print("type:", output_details[0]['dtype'])

tflite_interpreter_quant.allocate_tensors();

== Input details ==
name: x.3
shape: [    1     1 30225     1]
type: <class 'numpy.float32'>

== Output details ==
name: 185
shape: [1 2]
type: <class 'numpy.float32'>


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### loading dataset from npz format
1. ACDNet input length is 30225
2. sr is 44100 and 20000
3. need to convert 16K to 20000
### ACDNet Config Setting
#### Training Parameters
1. opt.batchSize = 64;
2. opt.weightDecay = 5e-4;
3. opt.momentum = 0.9;
4. opt.nEpochs = 2000;
5. opt.LR = 0.1;
6. opt.schedule = [0.3, 0.6, 0.9];
7. opt.warmup = 10; 
#### Basic Net Configuration
- nClasses = 50
- nFolds = 5
- splits = \[i for in range(1, nFolds + 1)\]
- sr = 20000
- inputLength = 30225
<br>ngth = 30225;
### How to convert 16K sound to 44.1K with python and sox

if using sox the command is as following: <br />
    sox old.wav -b 16 new.wav 
if using python you can do as following: <br />
    import soundfile
    
data, samplerate = soundfile.read('old.wav
    <br />)
soundfile.write('new.wav', data, samplerate, subtype='PCM_1
6')

In [6]:
def genDataTimeStr():
    return datetime.today().strftime('%Y-%m-%d %H:%M:%S').replace('-',"").replace(' ',"").replace(':',"");

In [7]:
def getFileList(srcDir,regex='.*\.wav'):
    results = os.listdir(srcDir)
    out_files = []
    cnt_files = 0
    for file in results:
        if os.path.isdir(os.path.join(srcDir, file)):
            out_files += getMp3List(os.path.join(srcDir, file))
        elif re.match(regex, file,  re.I):  # file.startswith(startExtension) or file.endswith(".txt") or file.endswith(endExtension):
            out_files.append(os.path.join(srcDir, file))
            cnt_files = cnt_files + 1
    return out_files

## sound preprocessing functions

In [8]:
_inputLen = 30225
_nCrops = 2
def preprocess_setup():
    funcs = []
    funcs += [U.padding( _inputLen// 2),
              U.normalize(32768.0),
              U.multi_crop(_inputLen, _nCrops)]

    return funcs

def preprocess(sound, funcs):
    for f in funcs:
        sound = f(sound)

    return sound;

In [9]:
_funcs = preprocess_setup()

### Read Test Wav File

In [10]:
positive_test_wavs = "./test_sounds/alarm_52/"
negative_test_wavs = "./test_sounds/other_sounds_99/"

p_wav_list = getFileList(positive_test_wavs);
n_wav_list = getFileList(negative_test_wavs);

print(f"total alarm sounds:{len(p_wav_list)} ");
print(f"total other sounds:{len(n_wav_list)}")
p_label = 52;
n_label = 99;

sounds = [];
labels = [];
## sound = preprocess(sound, _funcs)
for i in p_wav_list:
    sound = wavio.read(i).data.T[0]
    start = sound.nonzero()[0].min();
    end = sound.nonzero()[0].max();
    sound = sound[start: end + 1]  
    label = int(p_label);
    if len(sound)> 220500:
        sound = sound[:220500]
    sound = preprocess(sound, _funcs);
    sounds.append(sound);
    labels.append(label);
    
for j in n_wav_list:
    sound = wavio.read(j).data.T[0]
    start = sound.nonzero()[0].min();
    end = sound.nonzero()[0].max();
    sound = sound[start: end + 1]  
    label = int(n_label); 
    sound = np.float32(preprocess(sound, _funcs));
    sounds.append(sound);
    labels.append(label);

print(f"sounds length: {len(sounds)}");
print(f"labels length: {len(labels)}");

# sound = wavio.read(test_sound_file).data.T[0]
# start = sound.nonzero()[0].min()
# end = sound.nonzero()[0].max()
# sound = sound[start: end + 1]  # Remove silent sections
# label = 18 #int(os.path.splitext(test_sound_file)[0].split('-')[-1])

# if len(sound)> 220500:
#     sound = sound[:220500]

# ec50_sound1 =  wavio.read(ec50_18_sound).data.T[0]
# start_ec50 = ec50_sound1.nonzero()[0].min()
# end_ec50 = ec50_sound1.nonzero()[0].max()
# ec50_sound1 = ec50_sound1[start_ec50:end_ec50+1]
# ec50_18_label = 18

total alarm sounds:60 
total other sounds:107
sounds length: 167
labels length: 167


In [39]:
print(sounds)
print(labels)

[array([[ 0.        ,  0.        ,  0.        , ..., -0.19067383,
        -0.13583374,  0.07992554],
       [ 0.10272217, -0.00878906, -0.19595337, ...,  0.        ,
         0.        ,  0.        ]]), array([[ 0.        ,  0.        ,  0.        , ..., -0.13998413,
        -0.02075195,  0.10272217],
       [ 0.30545044,  0.32110596,  0.27816772, ...,  0.        ,
         0.        ,  0.        ]]), array([[ 0.        ,  0.        ,  0.        , ..., -0.01455688,
        -0.05999756,  0.01202393],
       [ 0.01501465,  0.00579834, -0.01550293, ...,  0.        ,
         0.        ,  0.        ]]), array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.02282715, -0.00119019, -0.0116272 , ...,  0.        ,
         0.        ,  0.        ]]), array([[ 0.        ,  0.        ,  0.        , ...,  0.09228516,
         0.16952515,  0.1210022 ],
       [-0.2772522 , -0.26135254, -0.1680603 , ...,  0.        ,
         0.        ,  0.  

In [40]:
# sound = preprocess(sound, _funcs)

In [12]:
print(sounds[0][1].shape)
print(type(labels[0]))

(30225,)
<class 'int'>


In [42]:
# sound = np.expand_dims(sound, axis=1)
# sound = np.expand_dims(sound, axis=3)
# print(sound.shape)

In [28]:
test_len = len(sounds);
correct_num = 0;
for w in range(test_len): 
    s = sounds[w]
    l = labels[w]
    s_test = np.expand_dims(s[0], axis=0);
    s_test = np.expand_dims(s_test, axis=1);
    s_test = np.expand_dims(s_test, axis=3);
    s_test = np.float32(s_test);
    # print(s_test.shape)
    # print(f"the {w+1} item's shape:\n  {s.shape}");
    # Run inference
    # print(f"s type:{type(s)}, and s shape:{s.shape}")
    tflite_interpreter_quant.set_tensor(input_details[0]['index'], s_test);
    tflite_interpreter_quant.invoke()
    pred = tflite_interpreter_quant.get_tensor(output_details[0]['index'])
    # print(f"Prediction result shape:{pred.shape}\n");
    print(f"Prediction result: {pred}, and true label: {l}")
    if l == 52:
        if pred[0][0] > pred[0][1]:
            correct_num += 1;
    if l == 99:
        if pred[0][0] < pred[0][1]:
            correct_num += 1;
print(f"final result is {100*(correct_num/test_len)}");

Prediction result: [[0.01854026 0.98145974]], and true label: 52
Prediction result: [[0.9980293  0.00197074]], and true label: 52
Prediction result: [[0.04396204 0.956038  ]], and true label: 52
Prediction result: [[0.2637651  0.73623496]], and true label: 52
Prediction result: [[0.0198274 0.9801726]], and true label: 52
Prediction result: [[9.991321e-01 8.678982e-04]], and true label: 52
Prediction result: [[0.73623496 0.2637651 ]], and true label: 52
Prediction result: [[0.2028374 0.7971626]], and true label: 52
Prediction result: [[0.7493076  0.25069234]], and true label: 52
Prediction result: [[9.999937e-01 6.295171e-06]], and true label: 52
Prediction result: [[0.3987692 0.6012308]], and true label: 52
Prediction result: [[0.23805805 0.76194197]], and true label: 52
Prediction result: [[0.44885498 0.551145  ]], and true label: 52
Prediction result: [[0.8929671  0.10703287]], and true label: 52
Prediction result: [[0.9801726 0.0198274]], and true label: 52
Prediction result: [[0.01

In [27]:
# scores = model.predict(sound, batch_size=len(sound), verbose=0);
# print(type(scores))
# print(scores.shape)

# for res in scores:
#     max_value = res.max()
#     max_index = np.argmax(res)
#     print(f"max value:{max_value:.5f} and index is {max_index}")
#     print('\n'.join('{}: {:.5f}'.format(*k) for k in enumerate(res)))